In [11]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [12]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, wait, worker_client
from vani.utils.dask_agg import nunique
from vani.utils.file_utils import ensure_dir
from vani.utils.json_encoders import NpEncoder
from vani.utils.logger import create_logger, format_log


In [13]:
client = Client(LocalCluster(n_workers=16, local_directory=f"/var/tmp/playground-quartz/local"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45960/status,
Dashboard: http://127.0.0.1:45960/status,Workers: 16
Total threads: 48,Total memory: 125.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43132,Workers: 16
Dashboard: http://127.0.0.1:45960/status,Total threads: 48
Started: Just now,Total memory: 125.50 GiB
Comm: tcp://127.0.0.1:45282,Total threads: 3
Dashboard: http://127.0.0.1:46521/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:44344,


In [14]:
CACHE_DIR = "cached"
METRICS_DIR = "metrics"
INDEX_DIR = "indexed"
XFER_SIZE_BINS = [
    -np.inf,
    4 * 1024.0,
    16 * 1024.0,
    64 * 1024.0,
    256 * 1024.0,
    1 * 1024.0 * 1024.0,
    4 * 1024.0 * 1024.0,
    16 * 1024.0 * 1024.0,
    64 * 1024.0 * 1024.0,
    np.inf
]
XFER_SIZE_BIN_LABELS = [
    '<4KB',
    '~16KB',
    '~64KB',
    '~256KB',
    '~1MB',
    '~4MB',
    '~16MB',
    '~64MB',
    '>64MB'
]
XFER_SIZE_BIN_NAMES = [
    '<4KB',
    '4KB',
    '16KB',
    '64KB',
    '256KB',
    '1MB',
    '4MB',
    '16MB',
    '64MB',
    '>64MB'
]


def compute_metrics_file_id(ddf: DataFrame, fg_index: str, log_dir: str):
    unique_filenames_d = unique_filenames_delayed(ddf=ddf, log_dir=log_dir,
                                                  dask_key_name=f"unique-filenames-{fg_index}")
    save_filenames_d = save_filenames_delayed(filenames=unique_filenames_d, log_dir=log_dir,
                                              dask_key_name=f"save-filenames-{fg_index}")
    metrics_d = metrics_filenames_delayed(ddf=ddf, filenames=unique_filenames_d,
                                          dask_key_name=f"metrics-{fg_index}")
    return [unique_filenames_d, save_filenames_d, [metrics_d]]


def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max


@delayed
def metrics_filenames_delayed(ddf: DataFrame, filenames: list, fg_index='file_id'):
    print('calculating metrics for filenames', len(filenames))
    tasks_d = []
    for filename in filenames:
        target_ddf_d = Analysis.target_ddf_delayed(ddf=ddf, start=filename, stop=filename,
                                                   dask_key_name=f"target-ddf-{fg_index}-{filename}")
        filter_d = filter_delayed(ddf=target_ddf_d, fg_index=fg_index, start=filename, stop=filename,
                                  dask_key_name=f"filter-{fg_index}-{filename}")
        flatten_d = flatten_delayed(filter_d, dask_key_name=f"flatten-{fg_index}-{filename}")
        tasks_d.append(flatten_d)
    print('num of tasks created', len(tasks_d))
    # with worker_client() as client:
    #     print('submitting on', client)
    #     metrics_f = client.compute(tasks_d)
    #     print('gathering tasks', len(metrics_f))
    #     metrics = client.gather(metrics_f)
    metrics = dask.compute(tasks_d)
    print('computed metrics', type(metrics))
    print('computed metrics', len(metrics))
    # print('computed metrics', metrics)
    return metrics


def read_and_index_logs(prefix: str, fg_index: str, log_dir: str, use_cache=True):
    dask_suffix = f"{prefix}-{fg_index}"
    fg_index_dir = f"{log_dir}/{INDEX_DIR}/{fg_index}"
    if use_cache and os.path.exists(f"{fg_index_dir}/_metadata"):
        ddf_d = read_parquet_delayed(log_dir=fg_index_dir, index=[fg_index],
                                     dask_key_name=f"read-parquet-{dask_suffix}")
        persisted_ddf_d = persist_ddf_delayed(ddf=ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
        return [ddf_d, persisted_ddf_d]
    ddf_d = read_parquet_delayed(log_dir=log_dir, dask_key_name=f"read-parquet-{dask_suffix}")
    indexed_ddf_d = set_ddf_index_delayed(ddf=ddf_d, fg_index=fg_index,
                                          dask_key_name=f"set-index-{dask_suffix}")
    persisted_ddf_d = persist_ddf_delayed(ddf=indexed_ddf_d, dask_key_name=f"persist-ddf-{dask_suffix}")
    partitioned_ddf_d = repartition_delayed(ddf=persisted_ddf_d,
                                            dask_key_name=f"repartition-ddf-{dask_suffix}")
    save_ddf_d = save_ddf_delayed(ddf=partitioned_ddf_d, log_dir=log_dir, fg_index=fg_index,
                                  dask_key_name=f"save-ddf-{dask_suffix}")
    return [ddf_d, indexed_ddf_d, persisted_ddf_d, save_ddf_d, partitioned_ddf_d]


@delayed
def read_parquet_delayed(log_dir: str, index: list = None):
    if index:
        print("Index specified", index)
        return dd.read_parquet(f"{log_dir}/*.parquet", calculate_divisions=True, index=index)
    print("Index not specified")
    return dd.read_parquet(f"{log_dir}/*.parquet", index=False)


@delayed
def repartition_delayed(ddf: DataFrame, partition_size='128MB'):
    return ddf.repartition(partition_size=partition_size)


@delayed
def persist_ddf_delayed(ddf: DataFrame):
    ddf = ddf.persist()
    wait(ddf)
    return ddf


@delayed
def save_ddf_delayed(ddf: DataFrame, log_dir: str, fg_index: str):
    ddf.to_parquet(f"{log_dir}/{INDEX_DIR}/{fg_index}")


@delayed
def save_filenames_delayed(filenames: list, log_dir: str):
    filenames = list(filenames)
    filenames.sort()
    with open(f"{log_dir}/filenames.json", "w") as file:
        json.dump(filenames, file, cls=NpEncoder)


@delayed
def set_ddf_index_delayed(ddf: DataFrame, fg_index: str):
    return ddf.set_index([fg_index])


@delayed
def unique_filenames_delayed(ddf: DataFrame, log_dir: str):
    if os.path.exists(f"{log_dir}/filenames.json"):
        with open(f"{log_dir}/filenames.json", "r") as file:
            unique_filenames = json.load(file)
    else:
        unique_filenames = ddf.index.unique().compute()
    return unique_filenames


In [15]:
%%time
log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf

CPU times: user 42.9 ms, sys: 11.6 ms, total: 54.5 ms
Wall time: 54.1 ms


,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [16]:
%%time
stat_agg = ddf \
    .groupby(['io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
stat_agg = stat_agg[stat_agg.index.isin([1, 2, 3])]
stat_agg['duration', 'per'] = stat_agg.div(stat_agg['duration', 'sum'].sum(), level=0)['duration', 'sum']
stat_agg['file_id', 'per'] = stat_agg.div(stat_agg['file_id', 'nunique'].max(), level=0)['file_id', 'nunique']
stat_agg['index', 'per'] = stat_agg.div(stat_agg['index', 'count'].sum(), level=0)['index', 'count']
stat_agg['proc_id', 'per'] = stat_agg.div(stat_agg['proc_id', 'nunique'].max(), level=0)['proc_id', 'nunique']
stat_agg['size', 'per'] = stat_agg.div(stat_agg['size', 'sum'].sum(), level=0)['size', 'sum']
stat_agg['bw', 'sum'] = stat_agg['size', 'sum'] / stat_agg['duration', 'sum']
stat_agg['bw', 'per'] = stat_agg.div(stat_agg['bw', 'sum'].sum(), level=0)['bw', 'sum']
stat_agg['xfer', 'max'] = pd.cut(stat_agg['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'min'] = pd.cut(stat_agg['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'mean'] = pd.cut(stat_agg['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['bw', 'sum'] = stat_agg['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
stat_agg['size', 'sum'] = stat_agg['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
stat_agg.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
stat_agg = stat_agg.reindex(sorted(stat_agg.columns), axis=1)
stat_agg


CPU times: user 584 ms, sys: 156 ms, total: 740 ms
Wall time: 1.79 s


acc_pat         app        bw            duration              file_id  \
           max min nunique       per       sum       per          sum nunique   
io_cat                                                                          
1            0   0       1  0.289382  0.96GB/s  0.137894   821.819641    1280   
2            0   0       1  0.243110  0.81GB/s  0.164140   978.242554    1280   
3            1   0       1  0.467508  1.55GB/s  0.697966  4159.727051    2561   

                 hostname  index           proc_id         rank      size  \
             per  nunique  count       per nunique  per nunique       per   
io_cat                                                                      
1       0.499805        1  12800  0.175516    1280  1.0    1280  0.098259   
2       0.499805        1  12800  0.175516    1280  1.0    1280  0.098259   
3       1.000000        1  47328  0.648969    1280  1.0    1280  0.803483   

                    xfer                
              sum    max   mean    min  
io_cat                                  
1        790.00GB  >64MB  ~64MB  ~64MB  
2        790.00GB  >64MB  ~64MB  ~64MB  
3       6460.00GB  >64MB  >64MB   <4KB

In [17]:
# importing the modules
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.transform import cumsum
import math

data = stat_agg.reset_index()
data['io_cat'] = ['read', 'write', 'metadata']
data['duration', 'angle'] = data['duration', 'per'] * 2*math.pi
data['duration', 'color'] = Category10[3]
data['duration', 'per_fmt'] = data['duration', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['size', 'angle'] = data['size', 'per'] * 2*math.pi
data['size', 'color'] = Category10[3]
data['size', 'per_fmt'] = data['size', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['index', 'angle'] = data['index', 'per'] * 2*math.pi
data['index', 'color'] = Category10[3]
data['index', 'per_fmt'] = data['index', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")

data

io_cat acc_pat         app        bw            duration               \
                max min nunique       per       sum       per          sum   
0      read       0   0       1  0.289382  0.96GB/s  0.137894   821.819641   
1     write       0   0       1  0.243110  0.81GB/s  0.164140   978.242554   
2  metadata       1   0       1  0.467508  1.55GB/s  0.697966  4159.727051   

  file_id            ...   xfer  duration                       size           \
  nunique       per  ...    min     angle    color per_fmt     angle    color   
0    1280  0.499805  ...  ~64MB  0.866414  #1f77b4  13.79%  0.617378  #1f77b4   
1    1280  0.499805  ...  ~64MB  1.031325  #ff7f0e  16.41%  0.617378  #ff7f0e   
2    2561  1.000000  ...   <4KB  4.385447  #2ca02c  69.80%  5.048430  #2ca02c   

              index                   
  per_fmt     angle    color per_fmt  
0   9.83%  1.102797  #1f77b4  17.55%  
1   9.83%  1.102797  #ff7f0e  17.55%  
2  80.35%  4.077592  #2ca02c  64.90%  

[3 rows x 30 columns]

In [18]:
from bokeh.layouts import column, row

def pie_chart(by: str):
        p = figure(height=200, width=300, title=f"{by} by I/O category", toolbar_location=None, 
                tooltips=f"@io_cat_: @{by}_per_fmt", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=0, radius=0.4,
                start_angle=cumsum(f'{by}_angle', include_zero=True), end_angle=cumsum(f'{by}_angle'),
                fill_color=f'{by}_color', line_color=None, legend_field='io_cat_', source=data)

        p.axis.axis_label = None
        p.axis.visible = False
        p.grid.grid_line_color = None

        return p

p_dur = pie_chart('duration')
p_ix = pie_chart('index')
p_size = pie_chart('size')

show(row([p_dur, p_ix, p_size]))

In [10]:
stat_agg_desc = stat_agg.describe(percentiles=[.9])
stat_agg_desc

acc_pat          app        bw  duration                   file_id  \
            max  min nunique       per       per          sum      nunique   
count  3.000000  3.0     3.0  3.000000  3.000000     3.000000     3.000000   
mean   0.333333  0.0     1.0  0.333333  0.333333  1986.596313  1707.000000   
std    0.577350  0.0     0.0  0.118480  0.316053  1883.610718   739.585695   
min    0.000000  0.0     1.0  0.243110  0.137894   821.819641  1280.000000   
50%    0.000000  0.0     1.0  0.289382  0.164140   978.242554  1280.000000   
90%    0.800000  0.0     1.0  0.431883  0.591200  3523.430176  2304.800000   
max    1.000000  0.0     1.0  0.467508  0.697966  4159.727051  2561.000000   

                hostname         index           proc_id         rank  \
            per  nunique         count       per nunique  per nunique   
count  3.000000      3.0      3.000000  3.000000     3.0  3.0     3.0   
mean   0.666537      1.0  24309.333333  0.333333  1280.0  1.0  1280.0   
std    0.288788      0.0  19934.750095  0.273348     0.0  0.0     0.0   
min    0.499805      1.0  12800.000000  0.175516  1280.0  1.0  1280.0   
50%    0.499805      1.0  12800.000000  0.175516  1280.0  1.0  1280.0   
90%    0.899961      1.0  40422.400000  0.554278  1280.0  1.0  1280.0   
max    1.000000      1.0  47328.000000  0.648969  1280.0  1.0  1280.0   

           size  
            per  
count  3.000000  
mean   0.333333  
std    0.407161  
min    0.098259  
50%    0.098259  
90%    0.662438  
max    0.803483

In [11]:
%%time
xfer_agg = ddf[ddf['io_cat'].isin([1, 2])] \
    .map_partitions(lambda df: df.assign(xfer=pd.cut(df['size'], XFER_SIZE_BINS, right=True))) \
    .groupby(['io_cat', 'xfer']) \
    .agg({'duration': [sum], 'size': [sum], 'index': ['count']}) \
    .compute() \
    .replace(0, np.nan) \
    .dropna()
xfer_agg


CPU times: user 266 ms, sys: 59.1 ms, total: 325 ms
Wall time: 1.42 s


duration          size    index
                                        sum           sum    count
io_cat xfer                                                       
1      (16777216.0, 67108864.0]  734.486633  6.764573e+11  11520.0
       (67108864.0, inf]          87.333023  1.717987e+11   1280.0
2      (16777216.0, 67108864.0]  765.892822  6.764573e+11  11520.0
       (67108864.0, inf]         212.349716  1.717987e+11   1280.0

In [12]:
def format_agg(df, add_xfer=True):
    df['bw', 'sum'] = df['size', 'sum'] / df['duration', 'sum']
    df['bw', 'per'] = df.div(df['bw', 'sum'].sum(), level=0)['bw', 'sum']
    df['duration', 'per'] = df.div(df['duration', 'sum'].sum(), level=0)['duration', 'sum']
    df['index', 'per'] = df.div(df['index', 'count'].sum(), level=0)['index', 'count']
    df['size', 'per'] = df.div(df['size', 'sum'].sum(), level=0)['size', 'sum']
    if add_xfer:
        df['xfer', 'max'] = pd.cut(df['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df['xfer', 'min'] = pd.cut(df['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df['xfer', 'mean'] = pd.cut(df['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
        df.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
    df['bw', 'fmt'] = df['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
    df['size', 'fmt'] = df['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
    df = df.reindex(sorted(df.columns), axis=1)
    return df

def filter_xfer_agg(xfer_agg, io_cat):
    xfer_agg_cat = xfer_agg.loc[(io_cat,)]
    xfer_agg_cat = format_agg(xfer_agg_cat, add_xfer=False)
    return xfer_agg_cat

xfer_agg_read = filter_xfer_agg(xfer_agg, io_cat=1)
xfer_agg_read

bw                          duration  \
                               fmt       per           sum       per   
xfer                                                                   
(16777216.0, 67108864.0]  0.86GB/s  0.318886  9.209934e+08  0.893732   
(67108864.0, inf]         1.83GB/s  0.681114  1.967168e+09  0.106268   

                                        index           size            \
                                 sum    count  per       fmt       per   
xfer                                                                     
(16777216.0, 67108864.0]  734.486633  11520.0  0.9  630.00GB  0.797468   
(67108864.0, inf]          87.333023   1280.0  0.1  160.00GB  0.202532   

                                        
                                   sum  
xfer                                    
(16777216.0, 67108864.0]  6.764573e+11  
(67108864.0, inf]         1.717987e+11

In [13]:
xfer_agg_write = filter_xfer_agg(xfer_agg, io_cat=2)
xfer_agg_write

bw                          duration  \
                               fmt       per           sum       per   
xfer                                                                   
(16777216.0, 67108864.0]  0.82GB/s  0.521921  8.832272e+08  0.782927   
(67108864.0, inf]         0.75GB/s  0.478079  8.090366e+08  0.217073   

                                        index           size            \
                                 sum    count  per       fmt       per   
xfer                                                                     
(16777216.0, 67108864.0]  765.892822  11520.0  0.9  630.00GB  0.797468   
(67108864.0, inf]         212.349716   1280.0  0.1  160.00GB  0.202532   

                                        
                                   sum  
xfer                                    
(16777216.0, 67108864.0]  6.764573e+11  
(67108864.0, inf]         1.717987e+11

In [22]:
DELTA = 0.0001
PERCENTILE = .95
PERCENTILE_FMT = f"{int(PERCENTILE * 100)}%"


In [15]:
%%time
tmin_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmin_min, tmid_max

CPU times: user 4.02 ms, sys: 4.03 ms, total: 8.05 ms
Wall time: 6.02 ms


(0, 337020899)

In [16]:
%%time
num_time_tasks = 2 ** 10
time_interval = math.ceil(tmid_max / num_time_tasks)
# time_interval = math.ceil(time_max / 60 / 60 / 60)
trange = np.arange(0, tmid_max, time_interval)
trange

CPU times: user 247 µs, sys: 0 ns, total: 247 µs
Wall time: 209 µs


array([        0,    329122,    658244, ..., 336033562, 336362684,
       336691806])

In [17]:
%%time
# ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

CPU times: user 12.6 ms, sys: 14.5 ms, total: 27.1 ms
Wall time: 24.2 ms


In [30]:
%%time
trange_agg = ddf \
    .groupby(['trange', 'io_cat']) \
    .agg({'duration':[sum]}) \
    .compute()
trange_agg.sort_values(('duration', 'sum'), ascending=False)  

CPU times: user 166 ms, sys: 49.8 ms, total: 216 ms
Wall time: 911 ms


,,duration
,,sum
trange,io_cat,
178,3,200.461044
177,3,193.962082
179,3,167.386047
284,0,162.171722
128,0,159.836716
...,...,...
234,0,0.000004
901,3,0.000003


In [31]:
trange_agg['duration', 'cumsum'] = trange_agg['duration', 'sum'].cumsum() / trange_agg['duration', 'sum'].sum()
# trange_agg_file['duration', 'delta'] = trange_agg_file['duration', 'cumsum'] / trange_agg_file['duration', 'cumsum'].shift()
trange_agg['duration', 'delta'] = trange_agg['duration', 'cumsum'].diff().fillna(trange_agg['duration', 'cumsum'])
trange_agg_cut =  trange_agg[trange_agg['duration', 'delta'] > DELTA]
trange_agg_cut = trange_agg_cut.reindex(sorted(trange_agg_cut.columns), axis=1)
trange_agg_cut

duration                      
                 cumsum     delta         sum
trange io_cat                                
83     0       0.006283  0.006283  126.961731
86     0       0.009203  0.002919   58.991016
87     0       0.012091  0.002889   58.366074
126    0       0.012476  0.000307    6.210055
127    0       0.014416  0.001940   39.192337
...                 ...       ...         ...
942    0       0.998619  0.000974   19.682636
959    0       0.999101  0.000117    2.362392
961    0       0.999335  0.000205    4.134205
964    0       0.999720  0.000297    5.996655
965    0       0.999928  0.000208    4.209144

[1028 rows x 3 columns]

In [33]:
trange_agg_cut_piv = trange_agg_cut.reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={0: 'other',  1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,other,read,write,metadata
trange,,,,
83,126.961731,0.0,0.0,0.0
86,58.991016,0.0,0.0,0.0
87,58.366074,0.0,0.0,0.0
126,6.210055,0.0,0.0,0.0
127,39.192337,0.0,0.0,0.0
...,...,...,...,...
942,19.682636,0.0,0.0,0.0
959,2.362392,0.0,0.0,0.0
961,4.134205,0.0,0.0,0.0


In [24]:
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [38]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10_3, Category10_4

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

p = figure(width=800, height=200)
p.vbar_stack(impacts, width=0.7, x="trange", source=source, color=Category10_4, legend_label=impacts)

p.y_range.start = 0
p.xgrid.grid_line_color = None
p.legend.location = "top_right"

show(p)

In [16]:
trange_agg_sum = trange_agg.groupby(level=0).sum().sort_values('duration', ascending=False)
trange_agg_sum

,duration,size,index
trange,,,
"(58254594, 58583716]",201.515778,1459617792,121
"(57925472, 58254594]",195.287552,3313500160,118
"(93141526, 93470648]",187.105423,3330277376,150
"(58583716, 58912838]",167.788239,3900702720,102
"(93470648, 93799770]",161.712540,4387241984,140
...,...,...,...
"(16785222, 17114344]",0.000000,0,0
"(285019652, 285348774]",0.000000,0,0
"(16456100, 16785222]",0.000000,0,0


In [17]:
trange_agg_sum_desc = trange_agg_sum.describe(percentiles=[PERCENTILE])
trange_agg_sum_desc

,duration,size,index
count,1023.000000,1.023000e+03,1023.000000
mean,19.751944,8.438792e+09,158.868035
std,25.896238,1.164218e+10,396.456878
min,0.000000,0.000000e+00,0.000000
50%,11.823395,3.850371e+09,107.000000
95%,61.774520,3.495701e+10,449.000000
max,201.515778,6.764573e+10,10254.000000


In [18]:
trange_agg_sum[trange_agg_sum['duration'] > 0].count()

duration    851
size        851
index       851
dtype: int64

In [19]:
trange_agg_sum[trange_agg_sum['duration'] > trange_agg_sum_desc.loc[PERCENTILE_FMT]['duration']].count()

duration    52
size        52
index       52
dtype: int64

In [20]:
trange_agg_cut = trange_agg_sum[trange_agg_sum['duration'] > trange_agg_sum_desc.loc[PERCENTILE_FMT]['duration']].sort_values('duration', ascending=False)
trange_agg_cut

,duration,size,index
trange,,,
"(58254594, 58583716]",201.515778,1459617792,121
"(57925472, 58254594]",195.287552,3313500160,118
"(93141526, 93470648]",187.105423,3330277376,150
"(58583716, 58912838]",167.788239,3900702720,102
"(93470648, 93799770]",161.712540,4387241984,140
"(41798494, 42127616]",159.836716,0,832
"(126711970, 127041092]",156.779205,17943232512,219
"(92812404, 93141526]",146.531281,1484783616,103
"(165548366, 165877488]",143.937943,6903824384,186


In [21]:
%%time
trange_agg_file = ddf[ddf['trange'].isin(trange_agg_cut.index)] \
    .groupby(['file_id','io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
#         'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
trange_agg_file = trange_agg_file.sort_values(('duration', 'sum'), ascending=False)
trange_agg_file = format_agg(trange_agg_file)
trange_agg_file

CPU times: user 329 ms, sys: 64 ms, total: 393 ms
Wall time: 1.7 s


acc_pat         app            bw                \
                               max min nunique           fmt           per   
file_id             io_cat                                                   
6142509188972423790 0            0   0       1      0.00GB/s  0.000000e+00   
5347710268757005579 3            0   0       1      0.00GB/s  0.000000e+00   
5347710267150565560 3            0   0       1      0.00GB/s  0.000000e+00   
5347710265229338281 3            0   0       1      0.00GB/s  0.000000e+00   
5347710268588044907 3            0   0       1      0.05GB/s  3.309226e-09   
...                            ...  ..     ...           ...           ...   
5347710265638769794 3            1   1       1  12335.53GB/s  8.223006e-04   
5347710267870141575 3            0   0       1  45230.26GB/s  3.015102e-03   
5347710267091019793 3            1   1       1  12335.53GB/s  8.223006e-04   
5347710266750616561 3            1   1       1  12335.53GB/s  8.223006e-04   
5347710266717072891 3            1   1       1  12335.53GB/s  8.223006e-04   

                                              duration              hostname  \
                                     sum           per          sum  nunique   
file_id             io_cat                                                     
6142509188972423790 0       0.000000e+00  6.536931e-01  3380.140869        1   
5347710268757005579 3       0.000000e+00  1.632309e-03     8.440403        1   
5347710267150565560 3       0.000000e+00  1.632117e-03     8.439411        1   
5347710265229338281 3       0.000000e+00  1.591653e-03     8.230181        1   
5347710268588044907 3       5.330322e+07  1.552193e-03     8.026138        1   
...                                  ...           ...          ...      ...   
5347710265638769794 3       1.324517e+13  3.674453e-10     0.000002        1   
5347710267870141575 3       4.856562e+13  3.674453e-10     0.000002        1   
5347710267091019793 3       1.324517e+13  3.674453e-10     0.000002        1   
5347710266750616561 3       1.324517e+13  3.674453e-10     0.000002        1   
5347710266717072891 3       1.324517e+13  3.674453e-10     0.000002        1   

                           index           proc_id    rank    size            \
                           count       per nunique nunique     fmt       per   
file_id             io_cat                                                     
6142509188972423790 0       5029  0.558096    1198    1198  0.00GB  0.000000   
5347710268757005579 3          1  0.000111       1       1  0.00GB  0.000000   
5347710267150565560 3          1  0.000111       1       1  0.00GB  0.000000   
5347710265229338281 3          1  0.000111       1       1  0.00GB  0.000000   
5347710268588044907 3          2  0.000222       1       1  0.40GB  0.001018   
...                          ...       ...     ...     ...     ...       ...   
5347710265638769794 3          1  0.000111       1       1  0.02GB  0.000060   
5347710267870141575 3          1  0.000111       1       1  0.09GB  0.000219   
5347710267091019793 3          1  0.000111       1       1  0.02GB  0.000060   
5347710266750616561 3          1  0.000111       1       1  0.02GB  0.000060   
5347710266717072891 3          1  0.000111       1       1  0.02GB  0.000060   

                                        xfer                
                                  sum    max   mean    min  
file_id             io_cat                                  
6142509188972423790 0               0   <4KB   <4KB   <4KB  
5347710268757005579 3               0   <4KB   <4KB   <4KB  
5347710267150565560 3               0   <4KB   <4KB   <4KB  
5347710265229338281 3               0   <4KB   <4KB   <4KB  
5347710268588044907 3       427819008  >64MB  >64MB   <4KB  
...                               ...    ...    ...    ...  
5347710265638769794 3        25165824  ~64MB  ~64MB  ~64MB  
5347710267870141575 3        92274688  >64MB  >64MB  >64MB  
5347710267091019793 3

In [22]:
trange_agg_file_desc = trange_agg_file.describe(percentiles=[PERCENTILE])
trange_agg_file_desc

acc_pat                  app            bw                \
               max          min nunique           per           sum   
count  1941.000000  1941.000000  1941.0  1.941000e+03  1.941000e+03   
mean      0.142195     0.012365     1.0  5.151984e-04  8.298535e+12   
std       0.349340     0.110536     0.0  1.596575e-03  2.571677e+13   
min       0.000000     0.000000     1.0  0.000000e+00  0.000000e+00   
50%       0.000000     0.000000     1.0  4.031035e-08  6.492971e+08   
95%       1.000000     0.000000     1.0  3.998926e-03  6.441253e+13   
max       1.000000     1.000000     1.0  1.394974e-02  2.246949e+14   

           duration              hostname        index               \
                per          sum  nunique        count          per   
count  1.941000e+03  1941.000000   1941.0  1941.000000  1941.000000   
mean   5.151984e-04     2.664007      1.0     4.642452     0.000515   
std    1.483648e-02    76.717041      0.0   114.111168     0.012664   
min    3.674453e-10     0.000002      1.0     1.000000     0.000111   
50%    1.900232e-05     0.098258      1.0     2.000000     0.000222   
95%    7.472663e-04     3.863993      1.0     5.000000     0.000555   
max    6.536931e-01  3380.140869      1.0  5029.000000     0.558096   

           proc_id         rank         size                
           nunique      nunique          per           sum  
count  1941.000000  1941.000000  1941.000000  1.941000e+03  
mean      1.616692     1.616692     0.000515  2.165825e+08  
std      27.169484    27.169484     0.000860  3.615461e+08  
min       1.000000     1.000000     0.000000  0.000000e+00  
50%       1.000000     1.000000     0.000160  6.710886e+07  
95%       1.000000     1.000000     0.002195  9.227469e+08  
max    1198.000000  1198.000000     0.007623  3.204448e+09

In [23]:
trange_agg_file_cut_dur = trange_agg_file[trange_agg_file[('duration', 'sum')] > trange_agg_file_desc.loc[PERCENTILE_FMT]['duration']['sum']]
trange_agg_file_cut_dur = trange_agg_file_cut_dur.sort_values(('duration', 'per'), ascending=False)
trange_agg_file_cut_dur

acc_pat         app        bw                \
                               max min nunique       fmt           per   
file_id             io_cat                                               
6142509188972423790 0            0   0       1  0.00GB/s  0.000000e+00   
5347710268757005579 3            0   0       1  0.00GB/s  0.000000e+00   
5347710267150565560 3            0   0       1  0.00GB/s  0.000000e+00   
5347710265229338281 3            0   0       1  0.00GB/s  0.000000e+00   
5347710268588044907 3            0   0       1  0.05GB/s  3.309226e-09   
...                            ...  ..     ...       ...           ...   
5347710264935727390 3            0   0       1  0.00GB/s  0.000000e+00   
5347710266934568296 3            0   0       1  0.00GB/s  0.000000e+00   
5347710265660452233 3            0   0       1  0.00GB/s  0.000000e+00   
5347710266570596840 3            0   0       1  0.00GB/s  0.000000e+00   
5347710267546213654 3            0   0       1  0.00GB/s  0.000000e+00   

                                          duration              hostname  \
                                     sum       per          sum  nunique   
file_id             io_cat                                                 
6142509188972423790 0       0.000000e+00  0.653693  3380.140869        1   
5347710268757005579 3       0.000000e+00  0.001632     8.440403        1   
5347710267150565560 3       0.000000e+00  0.001632     8.439411        1   
5347710265229338281 3       0.000000e+00  0.001592     8.230181        1   
5347710268588044907 3       5.330322e+07  0.001552     8.026138        1   
...                                  ...       ...          ...      ...   
5347710264935727390 3       0.000000e+00  0.000878     4.541443        1   
5347710266934568296 3       0.000000e+00  0.000748     3.869232        1   
5347710265660452233 3       0.000000e+00  0.000748     3.869090        1   
5347710266570596840 3       0.000000e+00  0.000748     3.866947        1   
5347710267546213654 3       0.000000e+00  0.000747     3.864759        1   

                           index           proc_id    rank    size            \
                           count       per nunique nunique     fmt       per   
file_id             io_cat                                                     
6142509188972423790 0       5029  0.558096    1198    1198  0.00GB  0.000000   
5347710268757005579 3          1  0.000111       1       1  0.00GB  0.000000   
5347710267150565560 3          1  0.000111       1       1  0.00GB  0.000000   
5347710265229338281 3          1  0.000111       1       1  0.00GB  0.000000   
5347710268588044907 3          2  0.000222       1       1  0.40GB  0.001018   
...                          ...       ...     ...     ...     ...       ...   
5347710264935727390 3          1  0.000111       1       1  0.00GB  0.000000   
5347710266934568296 3          1  0.000111       1       1  0.00GB  0.000000   
5347710265660452233 3          1  0.000111       1       1  0.00GB  0.000000   
5347710266570596840 3          1  0.000111       1       1  0.00GB  0.000000   
5347710267546213654 3          1  0.000111       1       1  0.00GB  0.000000   

                                        xfer               
                                  sum    max   mean   min  
file_id             io_cat                                 
6142509188972423790 0               0   <4KB   <4KB  <4KB  
5347710268757005579 3               0   <4KB   <4KB  <4KB  
5347710267150565560 3               0   <4KB   <4KB  <4KB  
5347710265229338281 3               0   <4KB   <4KB  <4KB  
5347710268588044907 3       427819008  >64MB  >64MB  <4KB  
...                               ...    ...    ...   ...  
5347710264935727390 3               0   <4KB   <4KB  <4KB  
5347710266934568296 3               0   <4KB   <4KB  <4KB  
5347710265660452233 3               0   <4KB   <4KB  <4KB  
5347710266570596840 3               0   <4KB   <4KB  <4KB  
5347710267546213654 3   

In [24]:
trange_agg_file_cut_bw = trange_agg_file[trange_agg_file[('bw', 'sum')] < trange_agg_file_desc.loc['50%']['bw']['sum']]
trange_agg_file_cut_bw = trange_agg_file_cut_bw.sort_values(('bw', 'per'), ascending=True)
trange_agg_file_cut_bw

acc_pat         app        bw                \
                               max min nunique       fmt           per   
file_id             io_cat                                               
6142509188972423790 0            0   0       1  0.00GB/s  0.000000e+00   
5347710266337416339 3            0   0       1  0.00GB/s  0.000000e+00   
5347710267742153139 3            0   0       1  0.00GB/s  0.000000e+00   
5347710266949348438 3            0   0       1  0.00GB/s  0.000000e+00   
5347710267438769842 3            0   0       1  0.00GB/s  0.000000e+00   
...                            ...  ..     ...       ...           ...   
5347710268290125637 1            0   0       1  0.60GB/s  3.993024e-08   
5347710266473797759 1            0   0       1  0.60GB/s  3.994029e-08   
5347710267187809523 2            0   0       1  0.60GB/s  3.999888e-08   
5347710267627761875 1            0   0       1  0.60GB/s  4.019169e-08   
5347710268434661767 2            0   0       1  0.60GB/s  4.023369e-08   

                                          duration              hostname  \
                                     sum       per          sum  nunique   
file_id             io_cat                                                 
6142509188972423790 0       0.000000e+00  0.653693  3380.140869        1   
5347710266337416339 3       0.000000e+00  0.000372     1.924382        1   
5347710267742153139 3       0.000000e+00  0.000372     1.924691        1   
5347710266949348438 3       0.000000e+00  0.000372     1.925979        1   
5347710267438769842 3       0.000000e+00  0.000373     1.926546        1   
...                                  ...       ...          ...      ...   
5347710268290125637 1       6.431746e+08  0.000040     0.208680        1   
5347710266473797759 1       6.433364e+08  0.000020     0.104314        1   
5347710267187809523 2       6.442801e+08  0.000020     0.104161        1   
5347710267627761875 1       6.473859e+08  0.000060     0.310984        1   
5347710268434661767 2       6.480623e+08  0.000020     0.103553        1   

                           index           proc_id    rank    size            \
                           count       per nunique nunique     fmt       per   
file_id             io_cat                                                     
6142509188972423790 0       5029  0.558096    1198    1198  0.00GB  0.000000   
5347710266337416339 3          1  0.000111       1       1  0.00GB  0.000000   
5347710267742153139 3          1  0.000111       1       1  0.00GB  0.000000   
5347710266949348438 3          1  0.000111       1       1  0.00GB  0.000000   
5347710267438769842 3          1  0.000111       1       1  0.00GB  0.000000   
...                          ...       ...     ...     ...     ...       ...   
5347710268290125637 1          2  0.000222       1       1  0.12GB  0.000319   
5347710266473797759 1          1  0.000111       1       1  0.06GB  0.000160   
5347710267187809523 2          1  0.000111       1       1  0.06GB  0.000160   
5347710267627761875 1          3  0.000333       1       1  0.19GB  0.000479   
5347710268434661767 2          1  0.000111       1       1  0.06GB  0.000160   

                                        xfer                
                                  sum    max   mean    min  
file_id             io_cat                                  
6142509188972423790 0               0   <4KB   <4KB   <4KB  
5347710266337416339 3               0   <4KB   <4KB   <4KB  
5347710267742153139 3               0   <4KB   <4KB   <4KB  
5347710266949348438 3               0   <4KB   <4KB   <4KB  
5347710267438769842 3               0   <4KB   <4KB   <4KB  
...                               ...    ...    ...    ...  
5347710268290125637 1       134217728  ~64MB  ~64MB  ~64MB  
5347710266473797759 1        67108864  ~64MB  ~64MB  ~64MB  
5347710267187809523 2        67108864  ~64MB  ~64MB  ~64MB  
5347710267627761875 1       201326592  ~64MB  ~64MB  ~64MB  
53477102684

In [25]:
%%time
trange_agg_proc = ddf[ddf['trange'].isin(trange_agg_cut.index)] \
    .groupby(['proc_id','io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        # 'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
trange_agg_proc = trange_agg_proc.sort_values(('duration', 'sum'), ascending=False)
trange_agg_proc = format_agg(trange_agg_proc)
trange_agg_proc

CPU times: user 345 ms, sys: 75.8 ms, total: 420 ms
Wall time: 1.86 s


acc_pat         app            bw            \
                               max min nunique           fmt       per   
proc_id             io_cat                                               
5117778030849213908 0            0   0       1      0.00GB/s  0.000000   
5117778030849344980 0            0   0       1      0.00GB/s  0.000000   
5117778030848820692 0            0   0       1      0.00GB/s  0.000000   
5117778030850262484 0            0   0       1      0.00GB/s  0.000000   
5117778030848493012 0            0   0       1      0.00GB/s  0.000000   
...                            ...  ..     ...           ...       ...   
5117778030882833876 3            1   1       1  12335.53GB/s  0.000822   
5117778030854718932 3            1   1       1  12335.53GB/s  0.000822   
5117778030851311060 0            0   0       1      0.00GB/s  0.000000   
5117778030866843092 0            0   0       1      0.00GB/s  0.000000   
5117778030901315028 0            0   0       1      0.00GB/s  0.000000   

                                              duration               file_id  \
                                     sum           per           sum nunique   
proc_id             io_cat                                                     
5117778030849213908 0       0.000000e+00  2.127993e-03  1.100351e+01       1   
5117778030849344980 0       0.000000e+00  2.126816e-03  1.099742e+01       1   
5117778030848820692 0       0.000000e+00  2.121150e-03  1.096812e+01       1   
5117778030850262484 0       0.000000e+00  2.119925e-03  1.096179e+01       1   
5117778030848493012 0       0.000000e+00  2.119191e-03  1.095799e+01       1   
...                                  ...           ...           ...     ...   
5117778030882833876 3       1.324517e+13  3.674453e-10  1.900000e-06       1   
5117778030854718932 3       1.324517e+13  3.674453e-10  1.900000e-06       1   
5117778030851311060 0       0.000000e+00  1.740530e-10  9.000000e-07       1   
5117778030866843092 0       0.000000e+00  1.740530e-10  9.000000e-07       1   
5117778030901315028 0       0.000000e+00  1.740530e-10  9.000000e-07       1   

                           hostname index              rank    size           \
                            nunique count       per nunique     fmt      per   
proc_id             io_cat                                                     
5117778030849213908 0             1     4  0.000444       1  0.00GB  0.00000   
5117778030849344980 0             1     4  0.000444       1  0.00GB  0.00000   
5117778030848820692 0             1     4  0.000444       1  0.00GB  0.00000   
5117778030850262484 0             1     4  0.000444       1  0.00GB  0.00000   
5117778030848493012 0             1     4  0.000444       1  0.00GB  0.00000   
...                             ...   ...       ...     ...     ...      ...   
5117778030882833876 3             1     1  0.000111       1  0.02GB  0.00006   
5117778030854718932 3             1     1  0.000111       1  0.02GB  0.00006   
5117778030851311060 0             1     1  0.000111       1  0.00GB  0.00000   
5117778030866843092 0             1     1  0.000111       1  0.00GB  0.00000   
5117778030901315028 0             1     1  0.000111       1  0.00GB  0.00000   

                                       xfer                
                                 sum    max   mean    min  
proc_id             io_cat                                 
5117778030849213908 0              0   <4KB   <4KB   <4KB  
5117778030849344980 0              0   <4KB   <4KB   <4KB  
5117778030848820692 0              0   <4KB   <4KB   <4KB  
5117778030850262484 0              0   <4KB   <4KB   <4KB  
5117778030848493012 0              0   <4KB   <4KB   <4KB  
...                              ...    ...    ...    ...  
5117778030882833876 3       25165824  ~64MB  ~64MB  ~64MB  
5117778030854718932 3       25165824  ~64MB  ~64MB  ~64MB  
5117778030851311060 0              0   <4KB   <4KB   <4KB  
5117778030866843092 0       

In [26]:
trange_agg_proc_desc = trange_agg_proc.describe(percentiles=[1-PERCENTILE,PERCENTILE])
trange_agg_proc_desc

acc_pat                  app           bw                \
               max          min nunique          per           sum   
count  3138.000000  3138.000000  3138.0  3138.000000  3.138000e+03   
mean      0.087954     0.007648     1.0     0.000319  5.133033e+12   
std       0.283273     0.087133     0.0     0.001280  2.062161e+13   
min       0.000000     0.000000     1.0     0.000000  0.000000e+00   
5%        0.000000     0.000000     1.0     0.000000  0.000000e+00   
50%       0.000000     0.000000     1.0     0.000000  0.000000e+00   
95%       1.000000     0.000000     1.0     0.002422  3.901982e+13   
max       1.000000     1.000000     1.0     0.013950  2.246949e+14   

           duration               file_id hostname        index               \
                per           sum nunique  nunique        count          per   
count  3.138000e+03  3.138000e+03  3138.0   3138.0  3138.000000  3138.000000   
mean   3.186743e-04  1.647813e+00     1.0      1.0     2.871574     0.000319   
std    4.643183e-04  2.400915e+00     0.0      0.0     2.330191     0.000259   
min    1.740530e-10  9.000000e-07     1.0      1.0     1.000000     0.000111   
5%     1.563576e-09  8.085000e-06     1.0      1.0     1.000000     0.000111   
50%    4.743607e-05  2.452842e-01     1.0      1.0     2.000000     0.000222   
95%    1.399016e-03  7.234086e+00     1.0      1.0     8.000000     0.000888   
max    2.127993e-03  1.100351e+01     1.0      1.0    17.000000     0.001887   

         rank         size                
      nunique          per           sum  
count  3138.0  3138.000000  3.138000e+03  
mean      1.0     0.000319  1.339664e+08  
std       0.0     0.000721  3.031652e+08  
min       1.0     0.000000  0.000000e+00  
5%        1.0     0.000000  0.000000e+00  
50%       1.0     0.000000  0.000000e+00  
95%       1.0     0.001716  7.214203e+08  
max       1.0     0.007623  3.204448e+09

In [27]:
trange_agg_proc_cut_dur = trange_agg_proc[trange_agg_proc[('duration', 'sum')] > trange_agg_proc_desc.loc[PERCENTILE_FMT]['duration']['sum']]
trange_agg_proc_cut_dur = trange_agg_proc_cut_dur.sort_values(('duration', 'per'), ascending=False)
trange_agg_proc_cut_dur

acc_pat         app        bw            duration  \
                               max min nunique       fmt  per  sum       per   
proc_id             io_cat                                                     
5117778030849213908 0            0   0       1  0.00GB/s  0.0  0.0  0.002128   
5117778030849344980 0            0   0       1  0.00GB/s  0.0  0.0  0.002127   
5117778030848820692 0            0   0       1  0.00GB/s  0.0  0.0  0.002121   
5117778030850262484 0            0   0       1  0.00GB/s  0.0  0.0  0.002120   
5117778030848493012 0            0   0       1  0.00GB/s  0.0  0.0  0.002119   
...                            ...  ..     ...       ...  ...  ...       ...   
5117778030902560212 0            0   0       1  0.00GB/s  0.0  0.0  0.001401   
5117778030848165332 0            0   0       1  0.00GB/s  0.0  0.0  0.001401   
5117778030849934804 0            0   0       1  0.00GB/s  0.0  0.0  0.001400   
5117778030855767508 0            0   0       1  0.00GB/s  0.0  0.0  0.001399   
5117778030907606484 0            0   0       1  0.00GB/s  0.0  0.0  0.001399   

                                      file_id hostname index            \
                                  sum nunique  nunique count       per   
proc_id             io_cat                                               
5117778030849213908 0       11.003507       1        1     4  0.000444   
5117778030849344980 0       10.997424       1        1     4  0.000444   
5117778030848820692 0       10.968122       1        1     4  0.000444   
5117778030850262484 0       10.961787       1        1     4  0.000444   
5117778030848493012 0       10.957994       1        1     4  0.000444   
...                               ...     ...      ...   ...       ...   
5117778030902560212 0        7.246135       1        1     5  0.000555   
5117778030848165332 0        7.243504       1        1     3  0.000333   
5117778030849934804 0        7.239648       1        1     3  0.000333   
5117778030855767508 0        7.235633       1        1     2  0.000222   
5117778030907606484 0        7.234165       1        1     2  0.000222   

                              rank    size           xfer              
                           nunique     fmt  per sum   max  mean   min  
proc_id             io_cat                                             
5117778030849213908 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030849344980 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030848820692 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030850262484 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030848493012 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
...                            ...     ...  ...  ..   ...   ...   ...  
5117778030902560212 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030848165332 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030849934804 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030855767508 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  
5117778030907606484 0            1  0.00GB  0.0   0  <4KB  <4KB  <4KB  

[157 rows x 19 columns]

In [28]:
trange_agg_proc_cut_bw = trange_agg_proc[trange_agg_proc[('bw', 'sum')] < trange_agg_proc_desc.loc['50%']['bw']['sum']]
trange_agg_proc_cut_bw = trange_agg_proc_cut_bw.sort_values(('bw', 'per'), ascending=True)
trange_agg_proc_cut_bw

Empty DataFrame
Columns: [(acc_pat, max), (acc_pat, min), (app, nunique), (bw, fmt), (bw, per), (bw, sum), (duration, per), (duration, sum), (file_id, nunique), (hostname, nunique), (index, count), (index, per), (rank, nunique), (size, fmt), (size, per), (size, sum), (xfer, max), (xfer, mean), (xfer, min)]
Index: []

In [44]:
%%time
file_agg = ddf[ddf['io_cat'].isin([1, 2, 3])] \
    .groupby(['file_id']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        # 'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'io_cat': [min, max],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
file_agg = file_agg.sort_values(('duration', 'sum'), ascending=False)
file_agg = format_agg(file_agg)
file_agg

CPU times: user 330 ms, sys: 80.4 ms, total: 411 ms
Wall time: 1.8 s


acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
file_id                                                                     
5347710267150565560       1   0       1  0.61GB/s  0.000297  6.505565e+08   
5347710266789296543       1   0       1  0.61GB/s  0.000298  6.534358e+08   
5347710265503319523       1   0       1  0.61GB/s  0.000298  6.537201e+08   
5347710268612693268       1   0       1  0.61GB/s  0.000299  6.550440e+08   
5347710268550015720       1   0       1  0.61GB/s  0.000300  6.574031e+08   
...                     ...  ..     ...       ...       ...           ...   
3341432784126556057       0   0       1  0.00GB/s  0.000000  0.000000e+00   
3341432781123988900       0   0       1  0.00GB/s  0.000000  0.000000e+00   
3341432783630115954       0   0       1  0.00GB/s  0.000000  0.000000e+00   
3341432781766529748       0   0       1  0.00GB/s  0.000000  0.000000e+00   
3341432780643150221       0   0       1  0.00GB/s  0.000000  0.000000e+00   

                         duration            hostname index  ... io_cat      \
                              per        sum  nunique count  ...    max min   
file_id                                                      ...              
5347710267150565560  1.739522e-03  10.367187        1    48  ...      3   1   
5347710266789296543  1.731858e-03  10.321506        1    48  ...      3   1   
5347710265503319523  1.731104e-03  10.317016        1    48  ...      3   1   
5347710268612693268  1.727605e-03  10.296165        1    48  ...      3   1   
5347710268550015720  1.721406e-03  10.259216        1    48  ...      3   1   
...                           ...        ...      ...   ...  ...    ...  ..   
3341432784126556057  3.406161e-09   0.000020        1     4  ...      3   3   
3341432781123988900  3.406161e-09   0.000020        1     4  ...      3   3   
3341432783630115954  3.154474e-09   0.000019        1     4  ...      3   3   
3341432781766529748  3.003462e-09   0.000018        1     4  ...      3   3   
3341432780643150221  2.667880e-09   0.000016        1     4  ...      3   3   

                    proc_id    rank    size                         xfer  \
                    nunique nunique     fmt       per         sum    max   
file_id                                                                    
5347710267150565560       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710266789296543       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710265503319523       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710268612693268       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710268550015720       1       1  6.28GB  0.000781  6744440832  >64MB   
...                     ...     ...     ...       ...         ...    ...   
3341432784126556057       1       1  0.00GB  0.000000           0   <4KB   
3341432781123988900       1       1  0.00GB  0.000000           0   <4KB   
3341432783630115954       1       1  0.00GB  0.000000           0   <4KB   
3341432781766529748       1       1  0.00GB  0.000000           0   <4KB   
3341432780643150221       1       1  0.00GB  0.000000           0   <4KB   

                                  
                      mean   min  
file_id                           
5347710267150565560  >64MB  <4KB  
5347710266789296543  >64MB  <4KB  
5347710265503319523  >64MB  <4KB  
5347710268612693268  >64MB  <4KB  
5347710268550015720  >64MB  <4KB  
...                    ...   ...  
3341432784126556057   <4KB  <4KB  
3341432781123988900   <4KB  <4KB  
3341432783630115954   <4KB  <4KB  
3341432781766529748   <4KB  <4KB  
3341432780643150221   <4KB  <4KB  

[2561 rows x 21 columns]

In [45]:
file_agg_desc = file_agg.describe(percentiles=[1-PERCENTILE,PERCENTILE])
file_agg_desc

acc_pat             app           bw                    duration  \
               max     min nunique          per           sum           per   
count  2561.000000  2561.0  2561.0  2561.000000  2.561000e+03  2.561000e+03   
mean      0.499805     0.0     1.0     0.000390  8.562588e+08  3.904725e-04   
std       0.500098     0.0     0.0     0.000453  9.943383e+08  4.544081e-04   
min       0.000000     0.0     1.0     0.000000  0.000000e+00  2.667880e-09   
5%        0.000000     0.0     1.0     0.000000  0.000000e+00  6.527076e-09   
50%       0.000000     0.0     1.0     0.000000  0.000000e+00  5.241224e-05   
95%       1.000000     0.0     1.0     0.001174  2.574573e+09  1.317420e-03   
max       1.000000     0.0     1.0     0.003355  7.356795e+09  1.739522e-03   

                   hostname        index               io_cat               \
               sum  nunique        count          per     max          min   
count  2561.000000   2561.0  2561.000000  2561.000000  2561.0  2561.000000   
mean      2.327134      1.0    28.476376     0.000390     3.0     2.000390   
std       2.708177      0.0   127.236706     0.001745     0.0     1.000195   
min       0.000016      1.0     4.000000     0.000055     3.0     1.000000   
5%        0.000039      1.0     4.000000     0.000055     3.0     1.000000   
50%       0.312366      1.0    48.000000     0.000658     3.0     3.000000   
95%       7.851547      1.0    48.000000     0.000658     3.0     3.000000   
max      10.367187      1.0  6368.000000     0.087319     3.0     3.000000   

           proc_id         rank         size                
           nunique      nunique          per           sum  
count  2561.000000  2561.000000  2561.000000  2.561000e+03  
mean      1.499414     1.499414     0.000390  3.370904e+09  
std      25.273521    25.273521     0.000391  3.372879e+09  
min       1.000000     1.000000     0.000000  0.000000e+00  
5%        1.000000     1.000000     0.000000  0.000000e+00  
50%       1.000000     1.000000     0.000000  0.000000e+00  
95%       1.000000     1.000000     0.000781  6.744441e+09  
max    1280.000000  1280.000000     0.000781  6.744441e+09

In [46]:
file_agg_cut_dur = file_agg[file_agg[('duration', 'sum')] > file_agg_desc.loc[PERCENTILE_FMT]['duration']['sum']].sort_values(('duration', 'sum'), ascending=False)
file_agg_cut_dur

acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
file_id                                                                     
5347710267150565560       1   0       1  0.61GB/s  0.000297  6.505565e+08   
5347710266789296543       1   0       1  0.61GB/s  0.000298  6.534358e+08   
5347710265503319523       1   0       1  0.61GB/s  0.000298  6.537201e+08   
5347710268612693268       1   0       1  0.61GB/s  0.000299  6.550440e+08   
5347710268550015720       1   0       1  0.61GB/s  0.000300  6.574031e+08   
...                     ...  ..     ...       ...       ...           ...   
5347710266287204140       1   0       1  0.79GB/s  0.000389  8.533376e+08   
5347710266386670578       1   0       1  0.80GB/s  0.000390  8.543279e+08   
5347710266096489346       1   0       1  0.80GB/s  0.000390  8.556367e+08   
5347710268815042761       1   0       1  0.80GB/s  0.000391  8.563738e+08   
5347710267677118324       1   0       1  0.80GB/s  0.000391  8.581229e+08   

                     duration            hostname index  ... io_cat      \
                          per        sum  nunique count  ...    max min   
file_id                                                  ...              
5347710267150565560  0.001740  10.367187        1    48  ...      3   1   
5347710266789296543  0.001732  10.321506        1    48  ...      3   1   
5347710265503319523  0.001731  10.317016        1    48  ...      3   1   
5347710268612693268  0.001728  10.296165        1    48  ...      3   1   
5347710268550015720  0.001721  10.259216        1    48  ...      3   1   
...                       ...        ...      ...   ...  ...    ...  ..   
5347710266287204140  0.001326   7.903602        1    48  ...      3   1   
5347710266386670578  0.001325   7.894441        1    48  ...      3   1   
5347710266096489346  0.001323   7.882365        1    48  ...      3   1   
5347710268815042761  0.001321   7.875580        1    48  ...      3   1   
5347710267677118324  0.001319   7.859528        1    48  ...      3   1   

                    proc_id    rank    size                         xfer  \
                    nunique nunique     fmt       per         sum    max   
file_id                                                                    
5347710267150565560       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710266789296543       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710265503319523       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710268612693268       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710268550015720       1       1  6.28GB  0.000781  6744440832  >64MB   
...                     ...     ...     ...       ...         ...    ...   
5347710266287204140       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710266386670578       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710266096489346       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710268815042761       1       1  6.28GB  0.000781  6744440832  >64MB   
5347710267677118324       1       1  6.28GB  0.000781  6744440832  >64MB   

                                  
                      mean   min  
file_id                           
5347710267150565560  >64MB  <4KB  
5347710266789296543  >64MB  <4KB  
5347710265503319523  >64MB  <4KB  
5347710268612693268  >64MB  <4KB  
5347710268550015720  >64MB  <4KB  
...                    ...   ...  
5347710266287204140  >64MB  <4KB  
5347710266386670578  >64MB  <4KB  
5347710266096489346  >64MB  <4KB  
5347710268815042761  >64MB  <4KB  
5347710267677118324  >64MB  <4KB  

[128 rows x 21 columns]

In [47]:
file_agg_cut_bw = file_agg[file_agg[('bw', 'sum')] < file_agg_desc.loc['5%']['bw']['sum']].sort_values(('bw', 'sum'), ascending=True)
file_agg_cut_bw

Empty DataFrame
Columns: [(acc_pat, max), (acc_pat, min), (app, nunique), (bw, fmt), (bw, per), (bw, sum), (duration, per), (duration, sum), (hostname, nunique), (index, count), (index, per), (io_cat, max), (io_cat, min), (proc_id, nunique), (rank, nunique), (size, fmt), (size, per), (size, sum), (xfer, max), (xfer, mean), (xfer, min)]
Index: []

[0 rows x 21 columns]

In [48]:
%%time
proc_agg = ddf[ddf['io_cat'].isin([1, 2, 3])] \
    .groupby(['proc_id']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'io_cat': [min, max],
        # 'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
proc_agg = proc_agg.sort_values(('duration', 'sum'), ascending=False)
proc_agg = format_agg(proc_agg)
proc_agg


CPU times: user 350 ms, sys: 78.7 ms, total: 428 ms
Wall time: 1.88 s


acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
5117778030853735892       1   0       1  0.61GB/s  0.000297  6.505407e+08   
5117778030853539284       1   0       1  0.61GB/s  0.000298  6.534232e+08   
5117778030854260180       1   0       1  0.61GB/s  0.000298  6.536733e+08   
5117778030853932500       1   0       1  0.61GB/s  0.000299  6.550173e+08   
5117778030855964116       1   0       1  0.61GB/s  0.000300  6.573889e+08   
...                     ...  ..     ...       ...       ...           ...   
5117778030899873236       1   0       1  3.66GB/s  0.001794  3.932760e+09   
5117778030900463060       1   0       1  4.07GB/s  0.001996  4.375436e+09   
5117778030909375956       1   0       1  4.12GB/s  0.002016  4.420724e+09   
5117778030898103764       1   0       1  4.26GB/s  0.002086  4.573874e+09   
5117778030859503060       1   0       1  6.85GB/s  0.003355  7.355119e+09   

                     duration            file_id hostname  ...     index  \
                          per        sum nunique  nunique  ...       per   
proc_id                                                    ...             
5117778030853735892  0.001740  10.367438       3        1  ...  0.000782   
5117778030853539284  0.001732  10.321704       3        1  ...  0.000782   
5117778030854260180  0.001731  10.317755       3        1  ...  0.000782   
5117778030853932500  0.001728  10.296584       3        1  ...  0.000782   
5117778030855964116  0.001721  10.259438       3        1  ...  0.000782   
...                       ...        ...     ...      ...  ...       ...   
5117778030899873236  0.000288   1.714938       3        1  ...  0.000782   
5117778030900463060  0.000259   1.541433       3        1  ...  0.000782   
5117778030909375956  0.000256   1.525642       3        1  ...  0.000782   
5117778030898103764  0.000247   1.474558       3        1  ...  0.000782   
5117778030859503060  0.000154   0.916972       3        1  ...  0.000782   

                    io_cat        rank    size                         xfer  \
                       max min nunique     fmt       per         sum    max   
proc_id                                                                       
5117778030853735892      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853539284      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030854260180      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853932500      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030855964116      3   1       1  6.28GB  0.000781  6744440832  >64MB   
...                    ...  ..     ...     ...       ...         ...    ...   
5117778030899873236      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030900463060      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030909375956      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030898103764      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030859503060      3   1       1  6.28GB  0.000781  6744440832  >64MB   

                                  
                      mean   min  
proc_id                           
5117778030853735892  >64MB  <4KB  
5117778030853539284  >64MB  <4KB  
5117778030854260180  >64MB  <4KB  
5117778030853932500  >64MB  <4KB  
5117778030855964116  >64MB  <4KB  
...                    ...   ...  
5117778030899873236  >64MB  <4KB  
5117778030900463060  >64MB  <4KB  
5117778030909375956  >64MB  <4KB  
5117778030898103764  >64MB  <4KB  
5117778030859503060  >64MB  <4KB  

[1280 rows x 21 columns]

In [49]:
proc_agg_desc = proc_agg.describe(percentiles=[1-PERCENTILE,PERCENTILE])
proc_agg_desc

acc_pat             app           bw                   duration  \
          max     min nunique          per           sum          per   
count  1280.0  1280.0  1280.0  1280.000000  1.280000e+03  1280.000000   
mean      1.0     0.0     1.0     0.000781  1.712976e+09     0.000781   
std       0.0     0.0     0.0     0.000326  7.137768e+08     0.000328   
min       1.0     0.0     1.0     0.000297  6.505407e+08     0.000154   
5%        1.0     0.0     1.0     0.000357  7.825710e+08     0.000356   
50%       1.0     0.0     1.0     0.000773  1.695651e+09     0.000667   
95%       1.0     0.0     1.0     0.001450  3.178967e+09     0.001446   
max       1.0     0.0     1.0     0.003355  7.355119e+09     0.001740   

                   file_id hostname        index               io_cat          \
               sum nunique  nunique        count          per     max     min   
count  1280.000000  1280.0   1280.0  1280.000000  1280.000000  1280.0  1280.0   
mean      4.656085     3.0      1.0    56.975000     0.000781     3.0     1.0   
std       1.957407     0.0      0.0     0.156186     0.000002     0.0     0.0   
min       0.916972     3.0      1.0    56.000000     0.000768     3.0     1.0   
5%        2.121583     3.0      1.0    57.000000     0.000782     3.0     1.0   
50%       3.977493     3.0      1.0    57.000000     0.000782     3.0     1.0   
95%       8.618330     3.0      1.0    57.000000     0.000782     3.0     1.0   
max      10.367438     3.0      1.0    57.000000     0.000782     3.0     1.0   

         rank          size                
      nunique           per           sum  
count  1280.0  1.280000e+03  1.280000e+03  
mean      1.0  7.812500e-04  6.744441e+09  
std       0.0  1.084626e-19  0.000000e+00  
min       1.0  7.812500e-04  6.744441e+09  
5%        1.0  7.812500e-04  6.744441e+09  
50%       1.0  7.812500e-04  6.744441e+09  
95%       1.0  7.812500e-04  6.744441e+09  
max       1.0  7.812500e-04  6.744441e+09

In [50]:
proc_agg_cut_dur = proc_agg[proc_agg[('duration', 'sum')] > proc_agg_desc.loc[PERCENTILE_FMT]['duration']['sum']].sort_values(('duration', 'sum'), ascending=False)
proc_agg_cut_dur

acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
5117778030853735892       1   0       1  0.61GB/s  0.000297  6.505407e+08   
5117778030853539284       1   0       1  0.61GB/s  0.000298  6.534232e+08   
5117778030854260180       1   0       1  0.61GB/s  0.000298  6.536733e+08   
5117778030853932500       1   0       1  0.61GB/s  0.000299  6.550173e+08   
5117778030855964116       1   0       1  0.61GB/s  0.000300  6.573889e+08   
...                     ...  ..     ...       ...       ...           ...   
5117778030888469972       1   0       1  0.72GB/s  0.000352  7.721259e+08   
5117778030852818388       1   0       1  0.72GB/s  0.000352  7.722698e+08   
5117778030922876372       1   0       1  0.72GB/s  0.000353  7.737344e+08   
5117778030921893332       1   0       1  0.72GB/s  0.000355  7.774175e+08   
5117778030923793876       1   0       1  0.72GB/s  0.000355  7.776427e+08   

                     duration            file_id hostname  ...     index  \
                          per        sum nunique  nunique  ...       per   
proc_id                                                    ...             
5117778030853735892  0.001740  10.367438       3        1  ...  0.000782   
5117778030853539284  0.001732  10.321704       3        1  ...  0.000782   
5117778030854260180  0.001731  10.317755       3        1  ...  0.000782   
5117778030853932500  0.001728  10.296584       3        1  ...  0.000782   
5117778030855964116  0.001721  10.259438       3        1  ...  0.000782   
...                       ...        ...     ...      ...  ...       ...   
5117778030888469972  0.001466   8.734898       3        1  ...  0.000782   
5117778030852818388  0.001465   8.733271       3        1  ...  0.000782   
5117778030922876372  0.001463   8.716739       3        1  ...  0.000782   
5117778030921893332  0.001456   8.675443       3        1  ...  0.000782   
5117778030923793876  0.001455   8.672931       3        1  ...  0.000782   

                    io_cat        rank    size                         xfer  \
                       max min nunique     fmt       per         sum    max   
proc_id                                                                       
5117778030853735892      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853539284      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030854260180      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853932500      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030855964116      3   1       1  6.28GB  0.000781  6744440832  >64MB   
...                    ...  ..     ...     ...       ...         ...    ...   
5117778030888469972      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030852818388      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030922876372      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030921893332      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030923793876      3   1       1  6.28GB  0.000781  6744440832  >64MB   

                                  
                      mean   min  
proc_id                           
5117778030853735892  >64MB  <4KB  
5117778030853539284  >64MB  <4KB  
5117778030854260180  >64MB  <4KB  
5117778030853932500  >64MB  <4KB  
5117778030855964116  >64MB  <4KB  
...                    ...   ...  
5117778030888469972  >64MB  <4KB  
5117778030852818388  >64MB  <4KB  
5117778030922876372  >64MB  <4KB  
5117778030921893332  >64MB  <4KB  
5117778030923793876  >64MB  <4KB  

[64 rows x 21 columns]

In [51]:
proc_agg_cut_bw = proc_agg[proc_agg[('bw', 'sum')] < proc_agg_desc.loc['5%']['bw']['sum']].sort_values(('bw', 'sum'), ascending=True)
proc_agg_cut_bw

acc_pat         app        bw                          \
                        max min nunique       fmt       per           sum   
proc_id                                                                     
5117778030853735892       1   0       1  0.61GB/s  0.000297  6.505407e+08   
5117778030853539284       1   0       1  0.61GB/s  0.000298  6.534232e+08   
5117778030854260180       1   0       1  0.61GB/s  0.000298  6.536733e+08   
5117778030853932500       1   0       1  0.61GB/s  0.000299  6.550173e+08   
5117778030855964116       1   0       1  0.61GB/s  0.000300  6.573889e+08   
...                     ...  ..     ...       ...       ...           ...   
5117778030888469972       1   0       1  0.72GB/s  0.000352  7.721259e+08   
5117778030852818388       1   0       1  0.72GB/s  0.000352  7.722698e+08   
5117778030922876372       1   0       1  0.72GB/s  0.000353  7.737344e+08   
5117778030921893332       1   0       1  0.72GB/s  0.000355  7.774175e+08   
5117778030923793876       1   0       1  0.72GB/s  0.000355  7.776427e+08   

                     duration            file_id hostname  ...     index  \
                          per        sum nunique  nunique  ...       per   
proc_id                                                    ...             
5117778030853735892  0.001740  10.367438       3        1  ...  0.000782   
5117778030853539284  0.001732  10.321704       3        1  ...  0.000782   
5117778030854260180  0.001731  10.317755       3        1  ...  0.000782   
5117778030853932500  0.001728  10.296584       3        1  ...  0.000782   
5117778030855964116  0.001721  10.259438       3        1  ...  0.000782   
...                       ...        ...     ...      ...  ...       ...   
5117778030888469972  0.001466   8.734898       3        1  ...  0.000782   
5117778030852818388  0.001465   8.733271       3        1  ...  0.000782   
5117778030922876372  0.001463   8.716739       3        1  ...  0.000782   
5117778030921893332  0.001456   8.675443       3        1  ...  0.000782   
5117778030923793876  0.001455   8.672931       3        1  ...  0.000782   

                    io_cat        rank    size                         xfer  \
                       max min nunique     fmt       per         sum    max   
proc_id                                                                       
5117778030853735892      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853539284      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030854260180      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030853932500      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030855964116      3   1       1  6.28GB  0.000781  6744440832  >64MB   
...                    ...  ..     ...     ...       ...         ...    ...   
5117778030888469972      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030852818388      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030922876372      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030921893332      3   1       1  6.28GB  0.000781  6744440832  >64MB   
5117778030923793876      3   1       1  6.28GB  0.000781  6744440832  >64MB   

                                  
                      mean   min  
proc_id                           
5117778030853735892  >64MB  <4KB  
5117778030853539284  >64MB  <4KB  
5117778030854260180  >64MB  <4KB  
5117778030853932500  >64MB  <4KB  
5117778030855964116  >64MB  <4KB  
...                    ...   ...  
5117778030888469972  >64MB  <4KB  
5117778030852818388  >64MB  <4KB  
5117778030922876372  >64MB  <4KB  
5117778030921893332  >64MB  <4KB  
5117778030923793876  >64MB  <4KB  

[64 rows x 21 columns]